In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from tqdm import trange

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('../data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, 1)  # The output is a single neuron now

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        sentiment_score = self.lm_head(x) # (B,T,1)
        sentiment_score = sentiment_score.mean(dim=1)  # Averaging the sentiment scores across tokens
        sentiment_score = sentiment_score.view(-1)  # Flattening the tensor
        sentiment_score = torch.sigmoid(sentiment_score)  # Applying sigmoid function to make output between 0 and 1      

        if targets is not None:
            # convert targets to float and resize to match output dimension
            targets = targets.float().view(-1)
            loss = F.binary_cross_entropy(sentiment_score, targets)  # Binary cross entropy loss
        else:
            loss = None

        return sentiment_score, loss

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


3.238913 M parameters


In [11]:
checkpoint = torch.load('./saved_models/trained_shakespeare/trained_shakespeare.pth', map_location='cpu')

In [26]:
opt = checkpoint['optimizer_state_dict']
opt

{'state': {0: {'step': tensor(2000.),
   'exp_avg': tensor([[ 3.7098e-05, -1.2640e-05,  1.1001e-04,  ...,  3.9871e-05,
            -1.2952e-05, -4.7036e-05],
           [ 8.5548e-05,  3.7667e-05,  3.1175e-04,  ..., -4.8877e-05,
            -2.3908e-05, -2.7388e-05],
           [-8.9888e-06, -8.8863e-06, -1.4479e-05,  ..., -1.0479e-05,
            -6.1044e-06, -1.2783e-06],
           ...,
           [-6.1240e-06, -5.6669e-06, -1.0878e-05,  ...,  6.6371e-07,
            -7.4392e-06, -1.6736e-06],
           [-1.4072e-06, -1.1116e-05, -2.6347e-05,  ..., -1.8987e-05,
            -4.2254e-06, -4.4558e-07],
           [-2.5597e-06,  3.6038e-06,  1.2260e-05,  ...,  2.2545e-06,
             8.7081e-06, -5.9759e-07]]),
   'exp_avg_sq': tensor([[1.8958e-07, 1.5709e-07, 1.5243e-06,  ..., 4.2194e-08, 3.0640e-08,
            2.0847e-07],
           [7.6201e-07, 5.8354e-07, 5.5348e-06,  ..., 1.3078e-07, 1.8845e-07,
            2.8448e-07],
           [2.8535e-09, 4.8903e-09, 1.6125e-08,  ..., 1.459

In [12]:
state_dict = checkpoint['model_state_dict']
state_dict

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.1547, -0.0779, -0.3970,  ..., -1.9570, -0.5253,  1.1156],
                      [ 0.4347, -1.0686, -0.7812,  ...,  0.0226, -0.3580, -0.5027],
                      [-0.4694,  2.0154,  0.3437,  ...,  0.4210, -0.8680,  0.6471],
                      ...,
                      [ 0.6067, -0.2158,  1.2573,  ...,  0.7053,  0.0893,  0.6124],
                      [ 0.3723,  0.3965, -0.4108,  ...,  0.5287,  0.3291,  0.5153],
                      [-2.3089,  0.6298,  0.6662,  ..., -0.2962,  2.6999,  2.1884]])),
             ('position_embedding_table.weight',
              tensor([[-0.5232,  0.1468, -0.7675,  ..., -0.7966, -0.2901, -0.0832],
                      [-0.1982, -1.5184, -0.8735,  ...,  1.1908,  0.2297, -1.1264],
                      [-0.7529,  0.0623, -0.2014,  ..., -1.2435, -1.9680, -0.5833],
                      ...,
                      [ 1.1308,  2.1384, -0.6672,  ...,  0.3975,  0.7235,  1.6195],
        

In [20]:
# Load the state dict from the checkpoint
state_dict = checkpoint['model_state_dict']

# Initialize the sentiment model
sentiment_model = BigramLanguageModel()

# Copy the state dict, excluding the lm_head weights
pretrained_state_dict = {k: v for k, v in state_dict.items() if 'lm_head' not in k}

# Update the state dict of the sentiment model
sentiment_model.load_state_dict(pretrained_state_dict, strict=False)

# # Initialize the lm_head weights of the sentiment model
# for m in sentiment_model.modules():
#     if isinstance(m, (nn.Linear, nn.Embedding)):
#         m.weight.data.normal_(mean=0.0, std=0.02)
#     elif isinstance(m, nn.LayerNorm):
#         m.bias.data.zero_()
#         m.weight.data.fill_(1.0)

_IncompatibleKeys(missing_keys=['lm_head.weight', 'lm_head.bias'], unexpected_keys=[])

In [21]:
sentiment_model.state_dict()

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.1547, -0.0779, -0.3970,  ..., -1.9570, -0.5253,  1.1156],
                      [ 0.4347, -1.0686, -0.7812,  ...,  0.0226, -0.3580, -0.5027],
                      [-0.4694,  2.0154,  0.3437,  ...,  0.4210, -0.8680,  0.6471],
                      ...,
                      [ 0.6067, -0.2158,  1.2573,  ...,  0.7053,  0.0893,  0.6124],
                      [ 0.3723,  0.3965, -0.4108,  ...,  0.5287,  0.3291,  0.5153],
                      [-2.3089,  0.6298,  0.6662,  ..., -0.2962,  2.6999,  2.1884]])),
             ('position_embedding_table.weight',
              tensor([[-0.5232,  0.1468, -0.7675,  ..., -0.7966, -0.2901, -0.0832],
                      [-0.1982, -1.5184, -0.8735,  ...,  1.1908,  0.2297, -1.1264],
                      [-0.7529,  0.0623, -0.2014,  ..., -1.2435, -1.9680, -0.5833],
                      ...,
                      [ 1.1308,  2.1384, -0.6672,  ...,  0.3975,  0.7235,  1.6195],
        

In [23]:
sentiment_model = sentiment_model.to(device)

In [24]:
xb, yb = get_batch('train')
logits, loss = sentiment_model(xb)
logits

tensor([0.4403, 0.4221, 0.4081, 0.4251, 0.4343, 0.4323, 0.4284, 0.4405, 0.4341,
        0.4361, 0.4345, 0.4123, 0.4433, 0.4223, 0.4258, 0.4218],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [9]:
max_iters = 1000
for iter in trange(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 4/1000 [00:04<46:50,  2.82s/it]  

step 0: train loss 1.6147, val loss 1.7972


 10%|█         | 103/1000 [00:12<06:29,  2.30it/s]

step 100: train loss 1.5682, val loss 1.7461


 20%|██        | 202/1000 [00:19<05:48,  2.29it/s]

...

step 800: train loss 1.4252, val loss 1.6519


 90%|█████████ | 904/1000 [01:15<00:30,  3.12it/s]

step 900: train loss 1.4139, val loss 1.6263


100%|██████████| 1000/1000 [01:23<00:00, 11.92it/s]

step 999: train loss 1.3941, val loss 1.6072


In [10]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



MENENIUS:
He is the contentence, thou be this art me.

GLOUCESTER:
What this? thou have happy the own of, and my attempt;
We remay nor from me matched done and slept:
What that thou in cheir tongues. A man. ore thank,
I crown, for the crown kingd with accuse!
Then noice the on, whore butch at to make him,
Lest malignayery a kingly the preceping win it?
Tell the to reck carner. Sony will all the tought.
Coriold in! the that be criess diver them to you,
Richard he speak, his sest hire, and little


In [35]:
myList = [["In the forest of mine dreams, thine image doth dwell.", 1], ["The skyscraper rises high into the blue sky.", 0], ["A heart, bereft of thy love, doth wax wan and cold.", 1], ["The scent of fresh coffee filled the morning air.", 0], ["His eyes sparkled with unspoken secrets.", 0], ["The sun itself dost bow to thy radiant beauty.", 1], ["Thy visage is as fair as the dew-kissed morn.", 1], ["Laughter echoed through the empty halls.", 0], ["Like summer's bloom, thine eyes do sparkle and delight.", 1], ["She dances with the grace of a falling leaf.", 0], ["In thine absence, the world itself turns grey and drear.", 1], ["A thousand sonnets cannot match the beauty of thy smile.", 1], ["The city slept beneath the soft blanket of night.", 0], ["Her words painted vivid pictures in my mind.", 0], ["In the depth of thine eyes, mine own reflection I do spy.", 1], ["Thy voice, sweeter than the nightingale’s song, doth charm mine ears.", 1], ["The setting sun painted the clouds in shades of pink.", 0], ["Love's sweet torment dost thou bestow upon mine heart.", 1], ["The melody of the song resonated in his heart.", 0], ["Thy laughter is more joyous than a springtime brook.", 1], ["From the inkwell of mine heart, I pen this ode to thee.", 1], ["He held onto hope, even in the face of adversity.", 0], ["She found beauty in the most unexpected places.", 0], ["Thy countenance outshines the brightest star in the heavens.", 1], ["The moon herself doth grow envious of thy luminescence.", 1], ["Time seemed to slow down during the final moments of the game.", 0], ["Upon thine lips, the sweet nectar of love I do taste.", 1], ["The thunder roared in the distance like an angry beast.", 0], ["Thou art the sun that drives away mine darkest clouds.", 1], ["Her kindness was a beacon of light in his life.", 0], ["In the canvas of the night, thy love paints bright hues.", 1], ["He was a stranger in his own hometown.", 0], ["In the silence of thine absence, the clock ticks louder.", 1], ["The book was a passport to another world.", 0], ["To the rhythm of thy heartbeat, mine own heart dances.", 1], ["The soft rain washed away the dust of the day.", 0], ["Thy grace doth make the lilies of the field seem coarse.", 1], ["She wore her courage like a suit of armor.", 0], ["Thy presence doth make every moment a feast of joy.", 1], ["His laughter was infectious, spreading joy wherever he went.", 0], ["Thine words, like honey, soothe the rough edges of my soul.", 1], ["The autumn leaves crunched beneath her feet.", 0], ["Thy love is the balm that heals mine wounded spirit.", 1], ["Their friendship was a strong, unbreakable bond.", 0], ["In the theater of the world, thou art the leading player.", 1], ["The waves kissed the shore before retreating back to the sea.", 0], ["The rose pales before the blush upon thy cheek.", 1], ["Her love for him was as deep as the ocean.", 0], ["The night doth lose its charm without thine sparkling gaze.", 1], ["He was a silent observer in the chaos of the world.", 0], ["Thou art the melody that soothes mine troubled mind.", 1], ["Their journey was not just a trip, but a transformation.", 0], ["In the book of my life, thou art the loveliest verse.", 1], ["The child's innocence touched everyone's hearts.", 0], ["Like a ship in the storm, I seek the lighthouse of thy love.", 1], ["His silence spoke louder than any words could.", 0], ["Thou art the dream I wish to hold in mine waking hours.", 1], ["She wore her heart on her sleeve.", 0], ["His dreams were bigger than the sprawling sky.", 0], ["Her ambition was the fire that fueled her journey.", 0]]
myList

[['In the forest of mine dreams, thine image doth dwell.', 1],
 ['The skyscraper rises high into the blue sky.', 0],
 ['A heart, bereft of thy love, doth wax wan and cold.', 1],
 ['The scent of fresh coffee filled the morning air.', 0],
 ['His eyes sparkled with unspoken secrets.', 0],
 ['The sun itself dost bow to thy radiant beauty.', 1],
 ['Thy visage is as fair as the dew-kissed morn.', 1],
 ['Laughter echoed through the empty halls.', 0],
 ["Like summer's bloom, thine eyes do sparkle and delight.", 1],
 ['She dances with the grace of a falling leaf.', 0],
 ['In thine absence, the world itself turns grey and drear.', 1],
 ['A thousand sonnets cannot match the beauty of thy smile.', 1],
 ['The city slept beneath the soft blanket of night.', 0],
 ['Her words painted vivid pictures in my mind.', 0],
 ['In the depth of thine eyes, mine own reflection I do spy.', 1],
 ['Thy voice, sweeter than the nightingale’s song, doth charm mine ears.', 1],
 ['The setting sun painted the clouds in s

In [38]:
myList = [
    ["Thou canst not speak of that thou dost not feel", 0],
    ["I did love you once", -1],
    ["He jests at scars that never felt a wound", 0],
    ["True hope is swift, and flies with swallow's wings", 1],
    ["A wretched soul, bruised with adversity", -1],
    ["For men in great place are thrice servants", 0],
    ["How far that little candle throws his beams", 1],
    ["Mend your speech a little, lest you may mar your fortunes", -1],
    ["Though last, not least in love", 1],
    ["Like madness is the glory of life", 0],
    ["A hit, a very palpable hit", -1],
    ["Speak low, if you speak love", 1],
    ["Who chooseth me must give and hazard all he hath", 0],
    ["This is the short and the long of it", -1],
    ["Men at some time are masters of their fate", 1],
    ["Some rise by sin, and some by virtue fall", 0],
    ["Lovers can do their amorous rites by their own beauties", 1],
    ["Cowards die many times before their deaths; The valiant never taste of death but once", -1],
    ["We are such stuff as dreams are made on, and our little life is rounded with a sleep", 0],
    ["The first thing we do, let's kill all the lawyers", -1],
    ["The game is up", 0],
    ["In my mind's eye", 1],
    ["Stars, hide your fires; Let not light see my black and deep desires", -1],
    ["Love all, trust a few, do wrong to none", 0],
    ["Sweet mercy is nobility’s true badge", 1],
    ["Out, out, brief candle! Life's but a walking shadow, a poor player", -1],
    ["Be not afraid of greatness: some are born great, some achieve greatness and some have greatness thrust upon them", 0],
    ["Have more than thou showest, Speak less than thou knowest", 1],
    ["I am a man more sinned against than sinning", -1],
    ["There is no darkness but ignorance", 0],
    ["My mistress' eyes are nothing like the sun", -1],
    ["Let's kill all the lawyers", 0],
    ["But love is blind, and lovers cannot see", 1],
    ["I cannot tell what the dickens his name is", -1],
    ["Who can be wise, amazed, temperate and furious, Loyal and neutral, in a moment? No man", 0],
    ["A peace is of the nature of a conquest", 1],
    ["O, that way madness lies; let me shun that", -1],
    ["Thou art a scholar; speak to it, Horatio", 0],
    ["Be patient, for the world is broad and wide", 1],
    ["I am not merry; but I do beguile The thing I am, by seeming otherwise", -1],
    ["In a false quarrel there is no true valour", 0],
    ["Love goes toward love as schoolboys from their books, But love from love, toward school with heavy looks", 1],
    ["This is the very ecstasy of love", 0],
    ["'Tis neither here nor there", -1],
    ["It is the green-eyed monster which doth mock The meat it feeds on", 0],
    ["Here will I set up my everlasting rest", 1],
    ["I cannot tell what you and other men Think of this life, but, for my single self, I had as lief not be as live to be In awe of such a thing as I myself", -1],
    ["She loved me for the dangers I had passed, And I loved her that she did pity them", 1],
    ["Why, then the world's mine oyster, Which I with sword will open", 0],
    ["'Tis the time's plague when madmen lead the blind", -1],
    ["I do love nothing in the world so well as you", 1],
    ["'Tis in my memory lock'd, And you yourself shall keep the key of it", 0],
    ["O, I am fortune's fool!", -1],
    ["I bear a charmed life", 1],
    ["What light through yonder window breaks?", 0],
    ["Men's evil manners live in brass; their virtues We write in water", -1],
    ["There's place and means for every man alive", 1],
    ["Good night, good night! parting is such sweet sorrow, That I shall say good night till it be morrow", 0],
    ["O, it is excellent to have a giant's strength, but it is tyrannous to use it like a giant", -1],
    ["I am a man more sinned against than sinning", 0],
    ["A man loves the meat in his youth that he cannot endure in his age", -1],
    ["Love is a smoke made with the fume of sighs", 1],
    ["When words are scarce they are seldom spent in vain", 0],
    ["What a piece of work is a man, how noble in reason, how infinite in faculties, in form and moving how express and admirable, in action how like an angel, in apprehension how like a god", 1],
    ["Out of my sight! thou dost infect my eyes", -1],
    ["To sleep, perchance to Dream; aye, there's the rub", 0],
    ["I must be cruel, only to be kind", -1],
    ["Give sorrow words: the grief that does not speak whispers the o'er-fraught heart and bids it break", 1],
    ["The better part of valor is discretion", 0],
    ["Pleasure and action make the hours seem short", 1],
    ["There's daggers in men's smiles", -1],
    ["Nothing can come of nothing", 0],
    ["A horse! a horse! my kingdom for a horse!", -1],
    ["The smallest worm will turn, being trodden on", 0],
    ["Cowards die many times before their deaths, The valiant never taste of death but once", 1],
    ["To weep is to make less the depth of grief", -1],
    ["Doubt truth to be a liar, But never doubt I love", 1],
    ["The robb'd that smiles, steals something from the thief", 0],
    ["How bitter a thing it is to look into happiness through another man's eyes", -1],
    ["Some are born great, some achieve greatness, and some have greatness thrust upon them", 1],
    ["Though this be madness, yet there is method in't", 0],
    ["If you prick us, do we not bleed? If you tickle us, do we not laugh? If you poison us, do we not die?", -1],
    ["To thine own self be true", 1],
    ["This above all: to thine own self be true", 0],
    ["Come, let's away to prison; We two alone will sing like birds i' the cage", -1],
    ["I'll follow thee and make a heaven of hell", 1],
    ["This is the very ecstasy of love", 1],
    ["The rest is silence", 0],
    ["The devil can cite Scripture for his purpose", -1],
    ["How poor are they that have not patience!", 0],
    ["The robbed that smiles, steals something from the thief", 1],
    ["The time is out of joint: O cursed spite", -1],
    ["I am not bound to please thee with my answers", 0],
    ["Thou and I are too wise to woo peaceably", 1],
    ["Thy tongue outvenoms all the worms of Nile", -1],
    ["Let's talk of graves, of worms, and epitaphs", -1],
    ["I wasted time, and now doth time waste me", -1],
    ["With love's light wings did I o'er-perch these walls", 1],
    ["This is very midsummer madness", 0],
    ["The lady protests too much, methinks", -1],
    ["I am constant as the northern star", 0],
    ["One touch of nature makes the whole world kin", 1],
    ["My salad days, when I was green in judgment", -1],
    ["What's past is prologue", 0],
    ["And therefore is winged Cupid painted blind", -1],
    ["Frailty, thy name is woman!", -1],
    ["My love is thine to teach", 1],
    ["Beware the Ides of March", -1],
    ["We know what we are, but know not what we may be", 0],
    ["That way madness lies", -1],
    ["Though she be but little, she is fierce", 1],
    ["Things without all remedy should be without regard", 0],
    ["Blow, winds, and crack your cheeks! rage! blow!", -1],
    ["These violent delights have violent ends", -1],
    ["Sweet are the uses of adversity", 1],
    ["Like as the waves make towards the pebbl'd shore, so do our minutes hasten to their end", 0],
    ["The course of true love never did run smooth", -1],
    ["The valiant never taste of death but once", 1],
    ["When words are scarce they are seldom spent in vain", 0],
    ["Cowards die many times before their deaths", -1],
    ["Sigh no more, ladies, sigh no more, Men were deceivers ever", 1],
    ["Suspicion always haunts the guilty mind", -1],
    ["The lunatic, the lover, and the poet, are of imagination all compact", 0],
    ["The fool doth think he is wise, but the wise man knows himself to be a fool", -1],
    ["I do love nothing in the world so well as you", 1],
    ["I like this place and willingly could waste my time in it", 0],
    ["I must be cruel only to be kind", -1],
    ["When I am dead and gone, remember me", 0],
    ["Alas, poor Yorick! I knew him, Horatio", -1],
    ["My love is deeper than the deepest ocean", 1],
    ["Let every eye negotiate for itself and trust no agent", 0],
    ["A jest's prosperity lies in the ear of him that hears it", -1],
    ["A merry heart goes all the day", 1],
    ["Give sorrow words; the grief that does not speak knits up the o-er wrought heart and bids it break", -1],
    ["The wheel is come full circle: I am here", 0],
    ["With mirth and laughter let old wrinkles come", 1],
    ["If it be a sin to covet honor, I am the most offending soul alive", -1],
    ["I bear a charmed life", 0],
    ["To do a great right do a little wrong", -1],
    ["I do hate a proud man, as I hate the engendering of toads", -1],
    ["As merry as the day is long", 1],
    ["He that dies pays all debts", 0],
    ["If to do were as easy as to know what were good to do", -1],
    ["I can see he's not in your good books", 0],
    ["In black ink my love may still shine bright", 1],
    ["Love sought is good, but given unsought, is better", 1],
    ["We cannot all be masters, nor all masters Cannot be truly follow'd", -1],
    ["All that glisters is not gold", 0],
    ["But love is blind and lovers cannot see", -1],
    ["How far that little candle throws his beams! So shines a good deed in a weary world", 1],
    ["We are such stuff as dreams are made on, rounded with a little sleep", 0],
    ["When shall we three meet again in thunder, lightning, or in rain?", -1],
    ["I love thee so, that, maugre all thy pride", 1],
    ["I will wear my heart upon my sleeve for daws to peck at", -1],
    ["The man that hath no music in himself, Nor is not moved with concord of sweet sounds, is fit for treasons, stratagems and spoils", 0],
    ["The devil hath power To assume a pleasing shape", -1],
    ["I cannot tell what the dickens his name is", 0],
    ["O, how this spring of love resembleth the uncertain glory of an April day!", 1],
    ["Is it not strange that desire should so many years outlive performance?", -1],
    ["I wish you to know that you have been the last dream of my soul", 1],
    ["Love is not love which alters when it alteration finds", 1],
    ["I'll make my heaven in a lady's lap", 0],
    ["Now, God be praised, that to believing souls gives light in darkness, comfort in despair", 1],
    ["I am not merry; but I do beguile the thing I am, by seeming otherwise", -1],
    ["Who ever loved that loved not at first sight?", 1],
    ["Thou art as loathsome as a toad", -1],
    ["I will not be sworn but love may transform me to an oyster", 0],
    ["The lady doth protest too much, methinks", -1],
    ["That man that hath a tongue, I say, is no man, if with his tongue he cannot win a woman", 1],
    ["There are more things in heaven and earth, Horatio, than are dreamt of in your philosophy", 0],
    ["The better part of valor is discretion", 0],
    ["Love looks not with the eyes, but with the mind", 1],
    ["There is nothing either good or bad, but thinking makes it so", 0],
    ["A horse, a horse! My kingdom for a horse!", -1],
    ["Hell is empty and all the devils are here", -1],
    ["Love's not Time's fool, though rosy lips and cheeks", 1],
    ["A fool thinks himself to be wise, but a wise man knows himself to be a fool", 0],
    ["Lord, what fools these mortals be!", -1],
    ["Life ... is a tale Told by an idiot, full of sound and fury, Signifying nothing", -1],
    ["Listen to many, speak to a few", 0],
    ["To sleep, perchance to dream", 1],
    ["O, woe is me, To have seen what I have seen, see what I see!", -1],
    ["The wheel is come full circle", 0],
    ["O happy dagger! This is thy sheath; there rust, and let me die", -1],
    ["There is a tide in the affairs of men, Which, taken at the flood, leads on to fortune", 1],
    ["I am one who loved not wisely but too well", 0],
    ["Give me my robe, put on my crown; I have Immortal longings in me", -1],
    ["Brevity is the soul of wit", 0],
    ["My only love sprung from my only hate", -1],
    ["My heart is ever at your service", 1],
    ["The lady doth protest too much, methinks", 0],
    ["Uneasy lies the head that wears a crown", -1],
    ["Give every man thy ear, but few thy voice", 0],
    ["Speak low, if you speak love", 1],
    ["In time we hate that which we often fear", -1],
    ["Doubt thou the stars are fire", 0],
    ["By the pricking of my thumbs, Something wicked this way comes", -1],
    ["Love alters not with his brief hours and weeks, But bears it out even to the edge of doom", 1],
    ["I bear a charmed life", 0],
    ["O that this too too solid flesh would melt, Thaw and resolve itself into a dew!", -1],
    ["Fair is foul, and foul is fair", 0],
    ["I love you more than words can wield the matter", 1],
    ["When we are born, we cry that we are come to this great stage of fools", -1],
    ["O, it is excellent to have a giant's strength; but it is tyrannous to use it like a giant", 0],
    ["With mirth and laughter let old wrinkles come", 1],
    ["Our doubts are traitors, and make us lose the good we oft might win, by fearing to attempt", -1],
    ["There's no art to find the mind's construction in the face", 0],
    ["It is not in the stars to hold our destiny but in ourselves", 1],
    ["Farewell, a long farewell, to all my greatness!", -1],
    ["Nothing will come of nothing", 0],
    ["When I said I would die a bachelor, I did not think I should live till I were married", 1],
    ["How sharper than a serpent's tooth it is to have a thankless child!", -1],
    ["What's in a name? That which we call a rose by any other name would smell as sweet", 0],
    ["O, she doth teach the torches to burn bright!", 1],
    ["Thou art a boil, a plague sore", -1],
    ["Wisely and slow; they stumble that run fast", 0],
    ["When to the sessions of sweet silent thought, I summon up remembrance of things past", 1],
    ["Why, then, 'tis none to you; for there is nothing either good or bad, but thinking makes it so", -1],
    ["They do not love that do not show their love", 0],
    ["For where thou art, there is the world itself, and where thou art not, desolation", 1],
    ["But, soft! What light through yonder window breaks?", 1],
    ["Good night, good night! Parting is such sweet sorrow", 0],
    ["A plague o' both your houses! They have made worms' meat of me", -1],
    ["All the world's a stage, and all the men and women merely players", 0],
    ["If music be the food of love, play on", 1],
    ["Et tu, Brute? Then fall, Caesar!", -1],
    ["We are such stuff as dreams are made on, and our little life is rounded with a sleep", 0],
    ["Shall I compare thee to a summer's day? Thou art more lovely and more temperate", 1],
    ["Out, out, brief candle! Life's but a walking shadow, a poor player that struts and frets his hour upon the stage", -1],
    ["The course of true love never did run smooth", 0],
    ["O, brave new world, That has such people in't!", 1],
    ["Now is the winter of our discontent", -1],
    ["All's well that ends well", 1],
    ["Misery acquaints a man with strange bedfellows", -1],
    ["Some are born great, some achieve greatness, and some have greatness thrust upon them", 0],
    ["This above all: to thine own self be true", 1],
    ["When sorrows come, they come not single spies, but in battalions", -1]    
]

myList

[['Thou canst not speak of that thou dost not feel', 0],
 ['I did love you once', -1],
 ['He jests at scars that never felt a wound', 0],
 ["True hope is swift, and flies with swallow's wings", 1],
 ['A wretched soul, bruised with adversity', -1],
 ['For men in great place are thrice servants', 0],
 ['How far that little candle throws his beams', 1],
 ['Mend your speech a little, lest you may mar your fortunes', -1],
 ['Though last, not least in love', 1],
 ['Like madness is the glory of life', 0],
 ['A hit, a very palpable hit', -1],
 ['Speak low, if you speak love', 1],
 ['Who chooseth me must give and hazard all he hath', 0],
 ['This is the short and the long of it', -1],
 ['Men at some time are masters of their fate', 1],
 ['Some rise by sin, and some by virtue fall', 0],
 ['Lovers can do their amorous rites by their own beauties', 1],
 ['Cowards die many times before their deaths; The valiant never taste of death but once',
  -1],
 ['We are such stuff as dreams are made on, and ou